In [25]:
#Syntax to download data through terminal: rsync -aP kkulbay@student-shell.sys.kth.se:/afs/kth.se/misc/csc/dept/tmh/corpora/tidigits/disc_4.2.1/tidigits/test/ ~/Desktop/KTH/DT2119_Speech_and_Speaker_Recognition/dt2119/DT2119_Speech/LAB3/tidigits/test

import numpy as np
import lab3_toolsSPYDER as tools
import lab3_proto as proto
import lab1_proto as proto1
#import lab1_implementation as proto1
import lab1_tools as tools1
import lab2_proto as proto2
import lab2_tools as tools2
import matplotlib.pyplot as plt

# 4.1 Target Class Definition

In [26]:
phoneHMMs = np.load('lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
# stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]

# np.save("stateList.npy",stateList)

#loading genereated list from above 
stateList = np.load("stateList.npy", allow_pickle=True).tolist()
# print(stateList)
# print(phoneHMMs.keys())

# 4.2 Forced Alignment

In [27]:

#lmfccEXAMPLE = np.load("lmfccEXAMPLE.npy", allow_pickle=True)

filename = 'tidigits/train/man/nw/z43a.wav'
samples, samplingrate = tools.loadAudio(filename)

lmfccEXAMPLE = proto1.mfcc(samples,samplingrate=samplingrate)


# plt.pcolormesh(lmfccEXAMPLE)
# plt.title("MFCCs test")
# plt.show()

wordTrans = list(tools.path2info(filename)[2])
assert wordTrans == ['z', '4', '3']
#-------
prondict = {} 
prondict['o'] = ['ow']
prondict['z'] = ['z', 'iy', 'r', 'ow']
prondict['1'] = ['w', 'ah', 'n']
prondict['2'] = ['t', 'uw']
prondict['3'] = ['th', 'r', 'iy']
prondict['4'] = ['f', 'ao', 'r']
prondict['5'] = ['f', 'ay', 'v']
prondict['6'] = ['s', 'ih', 'k', 's']
prondict['7'] = ['s', 'eh', 'v', 'ah', 'n']
prondict['8'] = ['ey', 't']
prondict['9'] = ['n', 'ay', 'n']

phoneTrans = proto.words2phones(wordTrans, prondict)
assert phoneTrans == ['sil', 'z', 'iy', 'r', 'ow', 'sp', 'f', 'ao', 'r', 'sp', 'th', 'r', 'iy', 'sp', 'sil']
#---------
utteranceHMM = proto2.concatHMMs(phoneHMMs, phoneTrans)
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans
                  for stateid in range(nstates[phone])]
# print(stateTrans)
# print(stateList)
# print(utteranceHMM.keys())
# utteranceHMM["name"]

#-----
lab3example = np.load("lab3_example.npz", allow_pickle=True)

#print(utteranceHMM)
#print(lab3example["example"])#.f.filename)


In [28]:
obsloglik = tools2.log_multivariate_normal_density_diag(lmfccEXAMPLE,utteranceHMM["means"],utteranceHMM["covars"])
vloglik, vpath = proto2.viterbi(obsloglik,np.log(utteranceHMM["startprob"]),np.log(utteranceHMM["transmat"][:-1,:-1]))

# print(vpath)
# print(len(vpath),len(stateTrans))
viterbiStateTrans = []
for i,stateID in enumerate(vpath):
    viterbiStateTrans.append(stateTrans[stateID])

CTRLviterbiStateTrans = ['sil_0', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_1', 'sil_2', 'z_0', 'z_0', 'z_0', 'z_0', 'z_1', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'z_2', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_1', 'iy_2', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'ow_0', 'ow_1', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'ow_2', 'f_0', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_1', 'f_2', 'ao_0', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_1', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'ao_2', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_0', 'th_1', 'th_1', 'th_1', 'th_2', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_0', 'r_1', 'r_2', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_0', 'iy_1', 'iy_1', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'iy_2', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_0', 'sil_1', 'sil_2']

assert CTRLviterbiStateTrans == viterbiStateTrans
tools.frames2trans(viterbiStateTrans, outfilename='z43a.lab')

/var/folders/q9/fn0bq0g17g1_dkmkvbvcxll00000gn/T/ipykernel_28268/3749334359.py:2: RuntimeWarning: divide by zero encountered in log
  vloglik, vpath = proto2.viterbi(obsloglik,np.log(utteranceHMM["startprob"]),np.log(utteranceHMM["transmat"][:-1,:-1]))


'0 0.01 sil_0\n0.01 0.19000000000000003 sil_1\n0.19000000000000003 0.20000000000000004 sil_2\n0.20000000000000004 0.24000000000000007 z_0\n0.24000000000000007 0.25000000000000006 z_1\n0.25000000000000006 0.36000000000000015 z_2\n0.36000000000000015 0.4400000000000002 iy_0\n0.4400000000000002 0.45000000000000023 iy_1\n0.45000000000000023 0.46000000000000024 iy_2\n0.46000000000000024 0.5600000000000003 r_0\n0.5600000000000003 0.5700000000000003 r_1\n0.5700000000000003 0.5800000000000003 r_2\n0.5800000000000003 0.5900000000000003 ow_0\n0.5900000000000003 0.6000000000000003 ow_1\n0.6000000000000003 0.6900000000000004 ow_2\n0.6900000000000004 0.7000000000000004 f_0\n0.7000000000000004 0.8100000000000005 f_1\n0.8100000000000005 0.8200000000000005 f_2\n0.8200000000000005 0.8300000000000005 ao_0\n0.8300000000000005 0.9700000000000006 ao_1\n0.9700000000000006 1.0800000000000007 ao_2\n1.0800000000000007 1.1100000000000008 r_0\n1.1100000000000008 1.1200000000000008 r_1\n1.1200000000000008 1.13000

# 4.3 Feature Extraction